In [ ]:
# MY SIMULATION 
import itertools
%matplotlib inline
import os
import random 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import vertex_model as model
# from vertex_model.run_select_nosaveinfo import run_simulation_INM, definecolors, run_simulation_no_INM
import itertools
import numpy as np
import matplotlib.pyplot as plt
import vertex_model as model
import vertex_model.initialisation as init
import vertex_model.characterization as crt
import vertex_model.save_data as save
from vertex_model.forces import TargetArea, Tension, Perimeter, Pressure
import os
import seaborn as sns
import warnings
import matplotlib.colors as colors
warnings.filterwarnings('ignore') #Don't show warnings
#from Gobal_Constant import dt, viscosity, t_G1, t_G2, t_S, A_c, J, pos_d, T1_eps, P, microns, time_hours, expansion_constant #file with necessary constants
from vertex_model.run_select import save_data
sns.set_style("whitegrid")
from scipy.spatial import Delaunay
import math
from scipy.spatial import ConvexHull

In [ ]:
def cell_vertices(mesh, id):   
    """
    Computes the coordinates of the vertices of a cell
    mesh: mesh object
    id: id of the cell whose vertices have to be computed 
    """ 
    cell_i = []
    for i in range(len(mesh.face_id_by_edge)):
        if mesh.face_id_by_edge[i] == id:
            cell_i.append(i)
    return cell_i
def centroid(vertexes_x, vertexes_y):   
     """
     Computes the centroid of a polygon
     vertexes_x: x coordinates of the vertices of a polygon 
     vertexes_y: y coordinates of the vertices of a polygon 
     """
     _len = len(vertexes_x)
     _x = sum(vertexes_x) / _len
     _y = sum(vertexes_y) / _len
     return(_x, _y)
def centroid_polygon(cell_area, vx, vy):
    n = len(vx)
    v_x = vx.tolist(); v_y = vy.tolist()
    v_x.append(v_x[0]); v_y.append(v_y[0])
    x = []; y = []
    for i in range(n):
        x = x + ((v_x[i]+v_x[i+1])*(v_x[i]*v_y[i+1]-v_x[i+1]*v_y[i]))
        y = y +  ((v_y[i]+v_y[i+1])*(v_x[i]*v_y[i+1]-v_x[i+1]*v_y[i]))
    x = x /(6*cell_area);     y = y /(6*cell_area); 
    return (x, y)
def mesh_centres(mesh):
    '''
    Compute the centres of the cells of a given mesh 
    '''
    centres_x =[]
    centres_y = [] 
    centres =np.zeros((mesh.n_face, 2))
    for i in range(mesh.n_face):       
       cell_i = cell_vertices(mesh, i) #vertices of the ith cell 
       area_i = mesh.area[i]
       if len(mesh.vertices[0][cell_i]) != 0:
        cell_center_i = centroid(mesh.vertices[0][cell_i], mesh.vertices[1][cell_i])
        centres_x.append(cell_center_i[0])
        centres_y.append(cell_center_i[1])
        #centres.append(np.array([cell_center_i[0],cell_center_i[1] ]))
        centres[i, 0] = cell_center_i[0]
        centres[i, 1] = cell_center_i[1]
    return centres_x, centres_y, centres

def circumcenter(vertices):
    """ 
    Computes the circumcentre of a polygon
    vertices: coordinates of the vertices of the polygon whose circumcentre has to be computed 
    """
    hull = ConvexHull(vertices)
    edges = hull.points[hull.simplices]

    circumcenters = []
    for edge in edges:
        x1, y1 = edge[0]
        x2, y2 = edge[1]
        x3, y3 = edge[2]

        A = np.array([[x2 - x1, y2 - y1], [x3 - x1, y3 - y1]])
        b = 0.5 * np.array([[(x2 - x1) ** 2 + (y2 - y1) ** 2], [(x3 - x1) ** 2 + (y3 - y1) ** 2]])
        try:
            circumcenter = np.linalg.solve(A, b)
            circumcenter = [circumcenter[0][0] + x1, circumcenter[1][0] + y1]
            circumcenters.append(circumcenter)
        except np.linalg.LinAlgError:
            pass

    circumcenter = np.mean(circumcenters, axis=0)
    return circumcenter



In [ ]:
#Global constants
dt=0.001            #time step
viscosity= 0.02  #viscosity*dv/dt = F
P= 0.0
expansion_constant = 1
Number_simulations = 50
K=1.0 #area elasticity
G= 0.04 #contractility of the cell
L=0.075 #line tensions
Lambda_0 = 0.68 #Line tensions can be reduced by increasing cell-cell adhesion or reducing actin- myosin contractility
t_end = 40
area_dimension= 177 #np.pi*(15/2.0)**2 #15 micrometros de diámetro

In [ ]:
# run simulation
def run(simulation,N_step,skip):
    N_step = math.floor(N_step)
    skip = math.floor(skip)
    iter =  itertools.islice(simulation,0,N_step,skip)
    return [cells.copy() for cells in itertools.islice(simulation,0,N_step,skip)]

#simulation without division
def basic_simulation(cells,force,dt=dt,T1_eps=0.04):
    expansion = np.array([0.0,0.0])
    while True:
#         print cells.properties['parent_group']
        cells.mesh , number_T1, edg_rem = cells.mesh.transition(T1_eps)
#         cells.properties['Gamma'][cells.mesh.face_ids == 50]=np.array([0.0])
        F = force(cells)/viscosity
#         expansion = 0.05*np.average(F*cells.mesh.vertices,1)*dt
        dv = dt*model.sum_vertices(cells.mesh.edges,F) 
#         cells.mesh = cells.mesh.moved(dv).scaled(1.0+ expansion)
        if hasattr(cells.mesh.geometry,'width'):
            expansion[0] = expansion_constant*np.average(F[0]*cells.mesh.vertices[0])*dt/(cells.mesh.geometry.width**2)
        if hasattr(cells.mesh.geometry,'height'): #Cylinder mesh doesn't have 'height' argument
            expansion[1] = np.average(F[1]*cells.mesh.vertices[1])*dt/(cells.mesh.geometry.height**2)
        cells.mesh = cells.mesh.moved(dv).scaled(1.0+expansion)
        yield cells

In [ ]:
def definecolors(cells):
    peach = '#eed5b7'
    light_blue ='#87cefa'
    pink = '#ffc0cb'
    light_green = '#98fb98'
    import matplotlib.colors as colors
    vv=sns.color_palette("hls", 10)
    v=[colors.rgb2hex(colorrgb) for colorrgb in vv]
    palette = np.array([light_green, pink,light_green,'g','r','g','m','c','',peach])
    palette = np.array([v[0],v[1],v[1], v[1],v[4],v[5],v[6],v[7],v[8],v[9],peach])
    colors = cells.properties['parent_group']
    return palette[colors]

In [ ]:
simulations = [] # to store the different simulations for the different values of noise
noise= 0.5
ncellup =18
ncellacross = 18 #quiero comprobar como aumenta de tiempo ext cuando aumento mutant cells
ncells = ncellacross*ncellup
# ecad_cells_proportion = 0.04
# ecad_cells = int(ecad_cells_proportion* ncells)
# id_ecad_cells = random.sample(range(ncells), ecad_cells); print(id_ecad_cells)
mutant_cells_vec = np.arange(0, 0.5, 0.025) #different proportions of mutant cells in the mesh
#mutant_cells_vec=[0.18]
ecad_cells_vec = []
for m in mutant_cells_vec:
    history_10={}
    ecad_cells = int(m * ncells)
    

    # ecad_cells_vec.append(id_ecad_cells)
    #ids_Gamma0 = random.sample(range(1, 100), 10)
    simulations_same_mutant=[]
    for i in range(3):
        #run simulation with the choosen parameters
        id_ecad_cells = random.sample(range(ncells), ecad_cells); 
        rand =  np.random.RandomState() #random number to choose Lambda
        params = [K,G,L]  # K=x[0],G=x[1],L=x[2]
        hex_centres = init.hexagonal_centres(ncellup, ncellacross, noise, rand= np.random.RandomState())
        # create a voronoi mesh with those centres 
        mesh = init.toroidal_ic_mesh(hex_centres)
        cells = model.Cells(mesh,properties={'K':K,'Gamma':G,'P':0.0,'boundary_P':P,'Lambda':L, 'Lambda_boundary':0.5, 'A0':1.0})
        force = TargetArea() + Tension() + Perimeter() + Pressure()
        step = int(50/dt)
        history_init_10= run(basic_simulation(cells,force),step,int(1/dt)) 
        expansion_constant = 1
        cells = history_init_10[-1].copy()
        cells.properties['parent_group'] = np.zeros(len(cells),dtype=int)
        if id_ecad_cells != []:
            cells.properties['parent_group'][np.where([cells.mesh.face_ids == index for index in id_ecad_cells])[1]] = 1
        cells.properties['Gamma']= np.array([G,0])[cells.properties['parent_group']]
        cells.properties['Lambda']= np.array([L,Lambda_0])[cells.properties['parent_group']]
        force = TargetArea() + Tension() + Perimeter() + Pressure()
        N_Step = math.floor(t_end/dt)
        history_10[i]= run(basic_simulation(cells,force),N_Step,.010/dt) 
        #save_data(i,history_10[i],'cells10')
        simulations_same_mutant.append(history_10[i])
        save.save_simulation(i, history_10[i], m,id_ecad_cells,"8sep", ncellacross, noise)
        print(f'{m} mutant cells, sim number {i}')

    simulations.append(simulations_same_mutant)  


In [ ]:
## Extrusion Time 
def t_extrusion(simulation, id_ecad_cells): 
    t = 0
    for time_step in simulation:
        extrused_cells = 0
        t = t+1
        area = time_step.mesh.area
        for a in area:
            if a == 0:
                extrused_cells= extrused_cells+1
        if extrused_cells == len(id_ecad_cells): 
            break
    return t

In [ ]:

time_ext = []; 
for i, sim in enumerate(simulations):
    time_ext.append(t_extrusion(sim[0],ecad_cells_vec[i]))
plt.plot(mutant_cells_vec, time_ext, '.-'); plt.title('Time extrusion of mutant cells')
plt.xlabel('percentage of mutant cells'); plt.ylabel('Time extrusion')



# Evolution of the mesh during all time steps of the simulation

In [ ]:
area_vec = []
length_distortion_vec = []
angle_distortion_vec = []
edges_length_vec = []
j = 0 #NUMBER OF THE SIMULATION WE AER STUDYING
sim = simulations[j]
r = len(sim[0])
r = time_ext[j] +50
t = int(r/10)
t=t_end
print(r)
i = 0
while i < r: 
    mesh_i = sim[0][i].mesh
    centres_x, centres_y, centres = mesh_centres(mesh_i)
    tri = Delaunay(centres)
    triangles = centres[tri.simplices]
    non_obtuse, non_obtuse_indices, obtuse, obtuse_indices = crt.triangle_sieve(triangles, np.pi / 10, np.pi - (np.pi / 10))
    trisieve = tri.simplices[non_obtuse_indices]
    #plt.triplot(centres[:,0], centres[:,1], trisieve)
    length_distortion_vec.append(np.mean(crt.length_distortion(non_obtuse)))
    angle_distortion_vec.append(np.mean(crt.angle_distortion(non_obtuse)))
    edges_length_vec.append(crt.mean_perimeter(non_obtuse))
    a, std = crt.mean_area_triangles(non_obtuse)
    area_vec.append(a)
    i = i+1
print(len(area_vec))


In [ ]:
t = np.arange(0,3000, 1)
fig, axs = plt.subplots(2, 2, figsize=(7, 7))
stop = time_ext[0] + 50
j =0
axs[0][0].plot(t[0:stop], length_distortion_vec[0:stop], label ='Length distortion')
axs[0][0].axvline(x=time_ext[0], color='green')
axs[0][0].set_title('Length distortion'); axs[0][0].set_xlabel('Percentage of mutant cells')

axs[0][1].plot(t[0:stop], angle_distortion_vec[0:stop], label = 'Angle distortion')
axs[0][1].axvline(x=time_ext[j], color='green')
axs[0][1].set_title('Angle distortion'); axs[0][0].set_xlabel('Percentage of mutant cells')

axs[1][0].plot(t[0:stop], edges_length_vec[0:stop]); axs[1][0].set_title('Edges length'); axs[0][0].set_xlabel('Percentage of mutant cells')
axs[1][0].axvline(x=time_ext[j], color='green')

axs[1][1].plot(t[0:stop], area_vec[0:stop]); axs[1][1].set_title('Area'); axs[0][0].set_xlabel('Percentage of mutant cells')
axs[1][1].axvline(x=time_ext[j], color='green')


In [ ]:
i=0
hh = simulations[i][0]
for cells in hh:
    #ids_nofronter = [1,15,30,6,98, 23,42,56,68,71]
    #cells.properties['parent_group'] = np.zeros(len(cells),dtype=int)
    #cells.properties['parent_group'][np.where([cells.mesh.face_ids == index for index in ids_nofronter])[1]] = 1
    cells.properties['color'] = definecolors(cells)
    cells.properties['parent_group'] = np.zeros(len(cells),dtype=int)
    #ids_Gamma0 = [1,15,30,6,98, 23,42,56,68,71]
    #cells.properties['parent_group'][np.where([cells.mesh.face_ids == index for index in ids_Gamma0])[1]] = 1
# definition of the frame for the interactive video
history_10_plot = hh[0:600]#+history1
fig=plt.figure()
fig.set_size_inches(6,6);
ax = fig.gca()
# initialization function: plot the background of each frame
def init_fig():
    ax = plt.figure();
    return (ax,)
# animation function. This is called sequentially
def animate_fig(i):
    cells_array=history_10_plot
    v_max = np.max((np.max(cells_array[-1].mesh.vertices), np.max(cells_array[0].mesh.vertices)))
    size = 2.0*v_max
    cells= history_10_plot[i]
    return model.draw(cells,ax,size)
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig,animate_fig, init_func=init_fig,
                               frames=(len(history_10_plot)))
HTML(anim.to_html5_video())

save data:

In [ ]:
root= r'C:\Users\natal\OneDrive\Documentos\MASTER\TFM\Simulations\ShapeIndex'
folder = '0.15_0.18'
path_folder = os.path.join(root, folder)
if not os.path.exists(path_folder):
        os.makedirs(path_folder)
ext_t = []; ext_t.append(time_ext[k])
# file_si = os.path.join(path_folder, 'shape_index.txt')
# np.savetxt(file_si, shape_indices, delimiter=',')
file_et = os.path.join(path_folder, 'extrusion_time.txt')
np.savetxt(file_et, ext_t, delimiter=',')
# file_co = os.path.join(path_folder, 'cells_out.txt')
# np.savetxt(file_co, cell_out, delimiter=',')
file_a = os.path.join(path_folder, 'area.txt')
np.savetxt(file_a, area_vec, delimiter=',')
file_e = os.path.join(path_folder, 'edges.txt')
np.savetxt(file_e, edges_length_vec, delimiter=',')
file_ad = os.path.join(path_folder, 'angle_distortion.txt')
np.savetxt(file_ad, angle_distortion_vec, delimiter=',')
file_ld = os.path.join(path_folder, 'length_distortion.txt')
np.savetxt(file_ld, length_distortion_vec, delimiter=',')
